# Pandas 데모 - Explarotary Data Analysis

## 라이브러리 import 및 설정

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import lightgbm as lgb
from matplotlib import pyplot as plt
from matplotlib import rcParams
import numpy as np
from pathlib import Path
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import seaborn as sns
import warnings
from sklearn.model_selection import KFold
from sklearn.model_selection import RandomizedSearchCV

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt
import math
from sklearn.metrics import confusion_matrix

from sklearn.preprocessing import StandardScaler, RobustScaler, PolynomialFeatures # 데이터 표준화

# lightgbm 모델
from lightgbm import LGBMClassifier, plot_importance

# normalization
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline

#feature Selection
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_selection import f_classif
import itertools
from sklearn.metrics import log_loss

In [4]:
rcParams['figure.figsize'] = (18, 9)
plt.style.use('fivethirtyeight')
pd.set_option('max_columns', 100)
pd.set_option("display.precision", 4)
warnings.simplefilter('ignore')

In [5]:
data_dir = Path('./input/')
trn_file = data_dir / 'whole_trn1.csv'
tst_file = data_dir / 'whole_tst1.csv'
# feature_file = data_dir / 'feature.csv'
sample_file = data_dir / 'sample_submission.csv'
submission_dir = Path('../submission')
target_col = 'class'
seed = 42

In [6]:
algo_name = 'lgb'
feature_name = 'feature'
model_name = f'{algo_name}_{feature_name}'

#feature_file = feature_dir / f'{feature_name}.csv'
submission_file = submission_dir / f'{model_name}.csv'

# HyperParameter Tuning

### 새로운 피처 데이터 불러오기

In [8]:
trn_df = pd.read_csv(trn_file, index_col=0)
trn_df

,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z,class,one_max,one_min,one_diff,one_std,one_sum,one_mean,dered_max,dered_min,dered_diff,dered_std,dered_sum,dered_mean,airmass_max,airmass_min,airmass_diff,airmass_std,airmass_sum,airmass_mean,one_dered_max_diff,one_dered_min_diff,one_dered_sum_diff,one_dered_std_diff,one_airmass_max_diff,one_airmass_min_diff,one_airmass_sum_diff,one_airmass_std_diff,dered_airmass_max_diff,dered_airmass_min_diff,dered_airmass_sum_diff,dered_airmass_std_diff,oneChar_quasar_UB_jester,...,nObserve_airmass_r_divide,nObserve_airmass_i_divide,nObserve_airmass_z_divide,nDetect_airmass_u_divide,nDetect_airmass_g_divide,nDetect_airmass_r_divide,nDetect_airmass_i_divide,nDetect_airmass_z_divide,airmass_u_airmass_g_divide,airmass_u_airmass_r_divide,airmass_u_airmass_i_divide,airmass_u_airmass_z_divide,airmass_g_airmass_r_divide,airmass_g_airmass_i_divide,airmass_g_airmass_z_divide,airmass_r_airmass_i_divide,airmass_r_airmass_z_divide,airmass_i_airmass_z_divide,PCA0,incPCA0,incPCA1,incPCA2,incPCA3,incPCA4,incPCA5,incPCA6,incPCA7,incPCA8,incPCA9,incPCA10,incPCA11,incPCA12,incPCA13,incPCA14,incPCA15,incPCA16,incPCA17,incPCA18,incPCA19,incPCA20,incPCA21,incPCA22,incPCA23,incPCA24,incPCA25,incPCA26,incPCA27,incPCA28,incPCA29,incPCA30
0,23.2640,20.3368,19.0095,17.6724,16.9396,-8.1086e-05,23.1243,20.2578,18.9551,17.6321,16.9089,18,18,1.1898,1.1907,1.1890,1.1894,1.1902,0.0,23.2640,16.9396,6.3243,2.4994,97.2223,19.4445,23.1243,16.9089,6.2153,2.4562,96.8782,19.3756,1.1907,1.1890,0.0017,6.7348e-04,5.9490,1.1898,0.1397,0.0307,0.3441,0.0432,22.0733,15.7506,91.2733,2.4987,21.9336,15.7200,90.9292,2.4555,1.3854,...,15.1390,15.1343,15.1234,15.1291,15.1174,15.1390,15.1343,15.1234,0.9992,1.0007,1.0003,0.9996,1.0014,1.0011,1.0004,0.9997,0.9990,0.9993,4.1012e+05,-4.1012e+05,-44588.1645,700.0365,-705.3297,-2043.2987,-1.3843,-0.7598,-80.9044,20.6263,-8.6331,4.3895,-7.7916,-14.0630,-0.4668,1.2869,-1.2112,-0.8189,0.2422,-0.6224,0.6692,-0.8760,2.6006,2.2680,1.7183,-0.5257,-2.2948,-0.9208,0.1165,-0.7321,2.0001,1.0806
1,15.0521,14.0620,13.4524,13.2684,13.1689,4.5061e-03,14.9664,14.0045,13.4114,13.2363,13.1347,1,1,1.2533,1.2578,1.2488,1.2510,1.2555,1.0,15.0521,13.1689,1.8832,0.7807,69.0039,13.8008,14.9664,13.1347,1.8317,0.7586,68.7533,13.7507,1.2578,1.2488,0.0091,3.5867e-03,6.2664,1.2533,0.0857,0.0343,0.2507,0.0222,13.7943,11.9202,62.7375,0.7771,13.7086,11.8859,62.4869,0.7550,-0.0674,...,0.8008,0.7994,0.7965,0.7979,0.7950,0.8008,0.7994,0.7965,0.9964,1.0036,1.0018,0.9982,1.0073,1.0055,1.0018,0.9982,0.9946,0.9964,-1.3453e+05,1.3453e+05,1329.3993,234.1073,-139.3329,141.9779,-1.4078,-1.1979,28.7459,-56.2464,18.1141,23.0452,-35.0625,28.0287,-13.4217,-8.6403,-4.3723,-3.1014,3.5773,-6.6401,0.8811,-3.1372,7.8264,0.5818,-3.0748,-0.4297,0.3455,0.6091,-1.0218,-0.0643,-0.3036,-1.2939
2,16.7864,15.8254,15.5363,15.3935,15.3500,4.7198e-04,16.6076,15.6866,15.4400,15.3217,15.2961,2,2,1.0225,1.0241,1.0210,1.0217,1.0233,0.0,16.7864,15.3500,1.4364,0.5934,78.8917,15.7783,16.6076,15.2961,1.3116,0.5462,78.3521,15.6704,1.0241,1.0210,0.0031,1.2343e-03,5.1126,1.0225,0.1787,0.0540,0.5396,0.0472,15.7623,14.3290,73.7791,0.5922,15.5835,14.2751,73.2395,0.5450,-0.0893,...,1.9589,1.9575,1.9545,1.9560,1.9529,1.9589,1.9575,1.9545,0.9984,1.0015,1.0008,0.9992,1.0031,1.0023,1.0008,0.9993,0.9977,0.9985,-2.0247e+05,2.0247e+05,1399.9650,333.2776,117.0187,206.6743,-1.6829,-1.1102,16.6353,-27.8060,-10.1510,-3.5526,-13.1135,11.3563,13.2489,3.8241,0.6886,-3.6781,0.2105,-0.7360,-0.3681,0.5018,2.7486,-3.0211,-1.6622,0.1794,0.1387,-0.2358,0.7250,0.1066,0.9698,0.3417
3,25.6606,21.1887,20.2212,19.8949,19.6346,5.8143e-06,25.3536,20.9947,20.0873,19.7947,19.5552,4,3,1.2054,1.2061,1.2049,1.2051,1.2057,0.0,25.6606,19.6346,6.0260,2.4968,106.6001,21.3200,25.3536,19.5552,5.7985,2.4085,105.7854,21.1571,1.2061,1.2049,0.0012,4.6906e-04,6.0272,1.2054,0.3070,0.0795,0.8147,0.0883,24.4546,18.4298,100.5730,2.4964,24.1476,18

In [9]:
y = trn_df['class']
y

0         0.0
1         1.0
2         0.0
3         0.0
4         0.0
         ... 
319995    2.0
319996    1.0
319997    0.0
319998    0.0
319999    0.0
Name: class, Length: 318854, dtype: float64

In [10]:
trn_df.drop('class', axis=1, inplace= True)
print(trn_df.shape, y.shape)

(318854, 635) (318854,)


In [11]:
tst_df = pd.read_csv(tst_file, index_col=0)
tst_df

,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z,one_max,one_min,one_diff,one_std,one_sum,one_mean,dered_max,dered_min,dered_diff,dered_std,dered_sum,dered_mean,airmass_max,airmass_min,airmass_diff,airmass_std,airmass_sum,airmass_mean,one_dered_max_diff,one_dered_min_diff,one_dered_sum_diff,one_dered_std_diff,one_airmass_max_diff,one_airmass_min_diff,one_airmass_sum_diff,one_airmass_std_diff,dered_airmass_max_diff,dered_airmass_min_diff,dered_airmass_sum_diff,dered_airmass_std_diff,oneChar_quasar_UB_jester,airmass_quasar_UB_jester,...,nObserve_airmass_r_divide,nObserve_airmass_i_divide,nObserve_airmass_z_divide,nDetect_airmass_u_divide,nDetect_airmass_g_divide,nDetect_airmass_r_divide,nDetect_airmass_i_divide,nDetect_airmass_z_divide,airmass_u_airmass_g_divide,airmass_u_airmass_r_divide,airmass_u_airmass_i_divide,airmass_u_airmass_z_divide,airmass_g_airmass_r_divide,airmass_g_airmass_i_divide,airmass_g_airmass_z_divide,airmass_r_airmass_i_divide,airmass_r_airmass_z_divide,airmass_i_airmass_z_divide,PCA0,incPCA0,incPCA1,incPCA2,incPCA3,incPCA4,incPCA5,incPCA6,incPCA7,incPCA8,incPCA9,incPCA10,incPCA11,incPCA12,incPCA13,incPCA14,incPCA15,incPCA16,incPCA17,incPCA18,incPCA19,incPCA20,incPCA21,incPCA22,incPCA23,incPCA24,incPCA25,incPCA26,incPCA27,incPCA28,incPCA29,incPCA30
320000,19.1431,18.0172,17.4733,17.1144,16.9263,1.2130e-01,19.0750,17.9550,17.4308,17.0838,16.9087,1,1,1.1563,1.1515,1.1613,1.1588,1.1539,19.1431,16.9263,2.2168,0.8902,88.6742,17.7348,19.0750,16.9087,2.1662,0.8708,88.4533,17.6907,1.1613,1.1515,9.8340e-03,3.8873e-03,5.7819,1.1564,0.0681,0.0176,0.2209,0.0194,17.9817,15.7748,82.8923,0.8863,17.9136,15.7572,82.6714,0.8669,0.0344,-0.8064,...,0.8611,0.8629,0.8666,0.8648,0.8684,0.8611,0.8629,0.8666,1.0042,0.9957,0.9979,1.0021,0.9915,0.9937,0.9979,1.0022,1.0064,1.0043,-1.2800e+05,1.2800e+05,1196.9230,235.6550,-139.1880,160.5277,-1.9126,-1.0842,44.8146,-0.9643,21.7936,-4.6636,2.5326,-1.7053,-3.5711,2.8203,0.4458,-0.3456,-0.2308,0.7622,-0.2065,-0.7161,0.1335,0.1387,0.0398,0.1172,0.2534,0.0211,0.2916,-0.2548,0.1511,0.0856
320001,18.7409,17.1705,16.3691,15.9865,15.7307,-6.6907e-05,15.0703,14.3098,14.3901,14.5160,14.6372,2,2,1.0076,1.0074,1.0080,1.0078,1.0075,18.7409,15.7307,3.0102,1.2140,83.9976,16.7995,15.0703,14.3098,0.7606,0.2986,72.9234,14.5847,1.0080,1.0074,5.9200e-04,2.3467e-04,5.0383,1.0077,3.6706,1.4209,11.0743,0.9154,17.7329,14.7233,78.9593,1.2138,14.0624,13.3024,67.8851,0.2984,0.3678,-0.8098,...,1.9842,1.9845,1.9851,1.9848,1.9853,1.9842,1.9845,1.9851,1.0003,0.9997,0.9998,1.0001,0.9994,0.9996,0.9999,1.0002,1.0005,1.0003,4.3461e+05,-4.3461e+05,-18062.1118,1116.7790,-865.0933,751.4398,-0.7363,-1.1075,16.4824,-27.0627,-10.1138,-2.0959,-17.4369,12.0993,19.8159,5.0769,-2.6335,19.9420,-2.7615,2.5154,-5.1153,-2.5450,-1.2671,0.6944,0.8139,1.1252,-0.0103,-1.5128,-0.4931,0.1845,-0.0257,-2.0442
320002,19.2980,17.8803,17.2221,16.8096,16.6352,1.1001e-01,19.1004,17.7726,17.1460,16.7497,16.5689,1,1,1.3809,1.3905,1.3715,1.3761,1.3856,19.2980,16.6352,2.6628,1.0788,87.8453,17.5691,19.1004,16.5689,2.5315,1.0229,87.3376,17.4675,1.3905,1.3715,1.8988e-02,7.5053e-03,6.9046,1.3809,0.1977,0.0664,0.5078,0.0559,17.9076,15.2637,80.9407,1.0713,17.7099,15.1974,80.4329,1.0154,0.2533,-0.8172,...,0.7291,0.7267,0.7217,0.7242,0.7192,0.7291,0.7267,0.7217,0.9931,1.0068,1.0034,0.9966,1.0138,1.0104,1.0035,0.9966,0.9898,0.9932,-1.2803e+05,1.2803e+05,1200.7489,234.4996,-139.4618,160.7233,-1.7725,-1.1019,44.3260,-13.1638,23.1327,14.0420,9.4443,-3.6513,-4.8411,-0.3012,-0.0282,0.9019,0.1247,0.2142,0.2060,-1.2618,0.2913,0.9610,0.0863,0.2402,-1.3960,0.1594,-0.1542,-0.0571,0.6402,-0.3683
320003,18.0775,17.1569,16.9041,16.8118,16.7955,2.9697e-05,17.9729,17.0735,16.8470,16.7700,16.7668,1,1,1.1408,1.1408,1.1409,1.1408,1.1408,18.0775,16.7955,1.2819,0.5387,85.7458,17.1492,17.9729,16.7668,1.2062,0.5113,85.4301,17.0860,1.1409,1.1408,1.1600e-04,4.6736e-05,5.7041,1.1408,0.1045,0.0288,0.3157,0.0274

In [23]:
from sklearn.feature_selection import SelectFromModel
#LGBM 모델을 기반으로 속성들 선택

select = SelectFromModel(LGBMClassifier(objective = "multiclass",
         metric= 'multi_logloss',
         num_class= 3,
         verbose=100,
         boosting_type= 'gbdt',
         learning_rate= 0.005,
         num_iterations= 500, 
         num_leaves= 512, 
         max_depth= 27,
         #min_data_in_leaf=1,
         #feature_fraction= 0.54, 
         #bagging_fraction= 0.37, 
         # bagging_freq= 3, 
         #min_child_samples = 89,
         force_col_wise = True
        ), threshold = 2000,
                        )
select.fit(trn_df,y)

train_sel = select.transform(trn_df)

test_sel = select.transform(tst_df)

[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.908583
[LightGBM] [Info] Total Bins 160011
[LightGBM] [Info] Number of data points in the train set: 318854, number of used features: 634
[LightGBM] [Info] Start training from score -0.982011
[LightGBM] [Info] Start training from score -2.011401
[LightGBM] [Info] Start training from score -0.710005
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 163 and max_depth = 27
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 16
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 365 and max_depth = 27
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 16
[LightGBM] [Warning] No further splits with positiv

[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 20
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 27
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 27
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 20
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 27
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 20
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 27
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 20
[Light

[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 27
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 19
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 19
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 27
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 19
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 27
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 22
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 20
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 27
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 19
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 19
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 27
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 22
[Light

[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 27
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 27
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 22
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 27
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 21
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 27
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 21
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 19
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 27
[Light

[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 20
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 22
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 27
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 20
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 27
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 19
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 27
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 20
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 22
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 27
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 27
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 19
[Light

...

[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 23
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 27
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 25
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 24
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 27
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 25
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 24
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 27
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 25
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 24
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 27
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 19
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 24
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 27
[Light

[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 24
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 22
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 27
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 25
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 26
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 27
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 24
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 27
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 27
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 27
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 26
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 27
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 24
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 26
[Light

[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 27
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 27
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 25
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 27
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 27
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 23
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 27
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 27
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 25
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 27
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 27
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 27
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 27
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 27
[Light

[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 27
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 27
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 27
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 27
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 27
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 27
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 27
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 27
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 27
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 27
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 27
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 27
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 27
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 27
[Light

[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 27
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 27
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 27
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 27
[LightGBM] [Debug] Trained a tree with leaves = 512 and max_depth = 27


In [15]:
SelectFromModel?

In [24]:
train_sel.shape

(318854, 102)

In [25]:
select.threshold_ 

2000.0

## train - simple split

In [28]:
feature_name = trn_df.columns.tolist()
print(y.shape, trn_df.shape, tst_df.shape)

(318854,) (318854, 635) (80000, 635)


In [42]:
X_trn, X_val, y_trn, y_val = train_test_split(train_sel, y, test_size=.01, random_state=seed)
print(X_trn.shape, X_val.shape, y_trn.shape, y_val.shape)

clf = lgb.LGBMClassifier(objective='multiclass',
                         n_estimators=12000,
                         num_leaves=512,
                         learning_rate=0.001,
                         min_child_samples=10,
                         subsample=.5,
                         subsample_freq=5,
                         colsample_bytree=.8,
                         random_state=seed,
                         max_depth = 27,
                        # matric = 'multi_logloss',
                         n_jobs=-1)
                        #boosting_type = 'goss')
clf.fit(X_trn, y_trn,
        eval_set=[(X_val, y_val)],
        eval_metric='multiclass',
        early_stopping_rounds=100)

p_val = clf.predict(X_val)


print(f'{accuracy_score(y_val, p_val) * 100:.4f}%')

(315665, 102) (3189, 102) (315665,) (3189,)
[LightGBM] [Warning] bagging_fraction is set=1, subsample=0.5 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] feature_fraction is set=0.3, colsample_bytree=0.8 will be ignored. Current value: feature_fraction=0.3
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=1 will be ignored. Current value: bagging_freq=2
[1]	valid_0's multi_logloss: 0.992865
Training until validation scores don't improve for 100 rounds
[2]	valid_0's multi_logloss: 0.991288
[3]	valid_0's multi_logloss: 0.989714
[4]	valid_0's multi_logloss: 0.988168
[5]	valid_0's multi_logloss: 0.986607
[6]	valid_0's multi_logloss: 0.985064
[7]	valid_0's multi_logloss: 0.983508
[8]	valid_0's multi_logloss: 0.981966
[9]	valid_0's multi_logloss: 0.980419
[10]	valid_0's multi_logloss: 0.978888
[11]	valid_0's multi_logloss: 0.977347
[12]	valid_0's multi_logloss: 0.97584
[13]	valid_0's multi_logloss: 0.974309
[14]	valid_0's multi_logloss: 0.972789
[15]	valid_0'

[199]	valid_0's multi_logloss: 0.748575
[200]	valid_0's multi_logloss: 0.747602
[201]	valid_0's multi_logloss: 0.746631
[202]	valid_0's multi_logloss: 0.745664
[203]	valid_0's multi_logloss: 0.744701
[204]	valid_0's multi_logloss: 0.743737
[205]	valid_0's multi_logloss: 0.742768
[206]	valid_0's multi_logloss: 0.741808
[207]	valid_0's multi_logloss: 0.740858
[208]	valid_0's multi_logloss: 0.739902
[209]	valid_0's multi_logloss: 0.738952
[210]	valid_0's multi_logloss: 0.738003
[211]	valid_0's multi_logloss: 0.73706
[212]	valid_0's multi_logloss: 0.736116
[213]	valid_0's multi_logloss: 0.735161
[214]	valid_0's multi_logloss: 0.734219
[215]	valid_0's multi_logloss: 0.733271
[216]	valid_0's multi_logloss: 0.732329
[217]	valid_0's multi_logloss: 0.731389
[218]	valid_0's multi_logloss: 0.730466
[219]	valid_0's multi_logloss: 0.729535
[220]	valid_0's multi_logloss: 0.728602
[221]	valid_0's multi_logloss: 0.727675
[222]	valid_0's multi_logloss: 0.726756
[223]	valid_0's multi_logloss: 0.725829
[

KeyboardInterrupt: 

In [40]:
p_val = clf.predict(test_sel)

In [41]:
#submission
Submission_set = pd.read_csv(sample_file, index_col=0)
Submission_set['class'] = p_val
print(Submission_set.shape)
print(Submission_set.head())
#class 값 카운팅
print(Submission_set[target_col].value_counts())

#저장하기
Submission_set.to_csv('./submission/final_sub.csv')

(80000, 1)
        class
id           
320000    2.0
320001    0.0
320002    2.0
320003    0.0
320004    2.0
2.0    40631
0.0    29978
1.0     9391
Name: class, dtype: int64


In [ ]:
#저장하기
Submission_set.to_csv('./submission/final_sub.csv')